In [1]:
import pandas as pd
import numpy as np
import re
import ast
import os
from utils import changing_id, return_list

In [2]:
pd.set_option("display.max_colwidth", None)

In [3]:
df = pd.read_csv(r"data_clean.csv")

In [4]:
cd ~\output_specs

C:\Users\hoang\output_specs


D:\conda\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
D:\conda\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 2EDA-789B

 Directory of C:\Users\hoang\output_specs

09/19/2025  12:48 PM    <DIR>          .
09/14/2025  03:44 PM    <DIR>          ..
09/05/2025  01:46 PM    <DIR>          .ipynb_checkpoints
09/08/2025  04:06 PM            52,088 B? nh? RAM, ? c?ng.csv
09/08/2025  04:06 PM            45,807 B? x? ly.csv
09/15/2025  01:40 PM            30,679 chip_clean.csv
09/08/2025  04:06 PM           137,904 C?ng k?t n?i & t¡nh nang m? r?ng.csv
09/15/2025  01:40 PM            78,497 data_clean.csv
09/08/2025  04:06 PM            36,337 D? ho? (GPU).csv
09/08/2025  04:06 PM            57,048 K¡ch thu?c - Kh?i lu?ng - Pin.csv
09/08/2025  04:06 PM            47,937 M…n h�nh.csv
09/08/2025  04:06 PM             9,634 rating_sales.csv
09/19/2025  12:48 PM            27,081 size_clean.csv
              10 File(s)        523,012 bytes
               3 Dir(s)  38,632,833,024 bytes free


In [6]:
size = pd.read_csv("Kích thước - Khối lượng - Pin.csv")
size

,Thông tin Pin,Hệ điều hành,Thời điểm ra mắt,Kích thước,Chất liệu,data_id
0,"3-cell Li-ion, 41 Wh",Windows 11 Home SL,2023,Dài 359.8 mm - Rộng 236 mm - Dày 18.6 mm - 1.59 kg,Vỏ nhựa,327098
1,"3-cell Li-ion, 42 Wh",Windows 11 Home SL,2023,Dài 360.3 mm - Rộng 232.5 mm - Dày 17.9 mm - 1.63 kg,Vỏ nhựa,311178
2,60 Wh,Windows 11 Home SL,2024,Dài 343.4 mm - Rộng 239.5 mm - Dày 17.9 mm - 1.63 kg,Vỏ nhựa - nắp lưng bằng kim loại,334442
3,"Li-Po, 53.8 Wh",macOS Sequoia,03/2025,Dài 304.1 mm - Rộng 215 mm - Dày 11.3 mm - 1.24 kg,Vỏ nhôm tái chế 100%,335362
4,"3-cell Li-ion, 41 Wh",Windows 11 Home SL + Office Home 2024 vĩnh viễn,2025,Dài 358.5 mm - Rộng 235.56 mm - Dày 18.99 mm - 1.9 kg,Vỏ nhựa,337470
...,...,...,...,...,...,...
400,57 Wh Li-ion,Windows 11 Pro,2025,Dài 312.8 mm - Rộng 217.65 mm - Dày 18.3 mm - 1.22 kg,Vỏ nhôm,342508
401,"Li-Polymer, 80 Wh",Windows 11 Home SL + Office Home 2024 vĩnh viễn,2025,Dài 344.9 mm - Rộng 255.35 mm - Dày 21.54 mm - 1.9 kg,Vỏ nhựa - Nắp lưng bằng nhôm,342517
402,"4-cell Li-Polymer, 80 Wh",Windows 11 Home SL + Office Home 2024 vĩnh viễn,2025,Dài 344.9 mm - Rộng 255.35 mm - Dày 21.54 mm - 1.9 kg,Nắp lưng và chiếu nghỉ tay bằng nhôm,342523
403,"6-cell Li-Po, 83 Wh",Windows 11 Home SL,2025,Dài 357.5 mm - Rộng 269 mm - Dày 25.4 mm - 2.43 kg,Vỏ nhựa,342721


In [7]:
try:
    size = changing_id(size)
    size = size.map(return_list)
    df = changing_id(df)
except:
    None

In [8]:
#### hành trình làm sạch size
### Thông tin pin -> clear toàn bộ, lấy số nằm gần với Wh nhất
### gói bundle trong hệ điều hành cũng nên drop, k có nhiều ý nghĩa, ex: mac OS thì chương trình khác với máy window nhiều
### thời điểm ra mắt cũng nên drop từ start->/ nếu len > 4
### kích thước thì thấy nên để màn hình bao nhiêu inch sẽ hợp lý hơn -> chỉ cần lấy trọng lượng là ok
### chất liệu cũng khá là nhiều dạng khác nhau, nhưng ở đây khá khó để gom nhóm lại nên sẽ để lại, cũng k mất thông tin mà cũng k dư thừa thông tin

In [9]:
size["Thời điểm ra mắt"].value_counts()

Thời điểm ra mắt
2024       153
2025       151
2023        63
2022         9
10/2024      7
03/2025      5
6/2022       1
2021         1
2020         1
Name: count, dtype: int64

In [10]:
size["Chất liệu"].value_counts()

Chất liệu
Vỏ nhựa                                        212
Vỏ nhựa - nắp lưng bằng kim loại                64
Vỏ kim loại - Nhôm                              41
Vỏ kim loại                                     34
Vỏ nhôm tái chế 100%                            12
Nắp lưng và chiếu nghỉ tay bằng kim loại         7
Vỏ nhựa - chiếu nghỉ tay bằng kim loại           7
Vỏ kim loại - Nắp lưng bằng nhựa                 5
Vỏ hợp kim Nhôm                                  5
Nhôm - Magie                                     4
Vỏ nhựa - nắp lưng bằng hợp kim nhôm             3
Vỏ nhôm                                          3
Vỏ nhựa - Nắp lưng bằng nhôm                     2
Mặt trên Sợi Carbon - Mặt dưới nhôm              2
Vỏ bằng kim loại - Chiếu nghỉ tay bằng nhựa      1
Nắp lưng và mặt đáy bằng kim loại                1
Vỏ kim loại nguyên khối                          1
Nắp lưng và chiếu nghỉ tay bằng nhôm             1
Name: count, dtype: int64

In [11]:
### thông tin Pin
# pattern = r"(\d+)\s*Wh"

In [12]:
size["Thông tin Pin"].value_counts()

Thông tin Pin
3-cell Li-ion, 41 Wh        47
4-cell, 54Wh                27
3-cell Li-ion, 42 Wh        25
60 Wh                       22
3-cell, 50Wh                18
                            ..
6-cell Li-ion, 71 Wh         1
57 Wh Li-ion                 1
Li-Polymer, 80 Wh            1
4-cell Li-Polymer, 80 Wh     1
6-cell Li-Po, 83 Wh          1
Name: count, Length: 87, dtype: int64

In [13]:
pattern_battery = r"(\d+(?:\.\d+)?)(?=\s*Wh)"

def clean_1(x):
    """
    Trích xuất dung lượng pin theo định dạng "<số> (Wh)".

    Parameters
    ----------
    x : str
        Chuỗi mô tả pin.

    Returns
    -------
    str or None
        Chuỗi dung lượng kèm đơn vị Wh (ví dụ "41 Wh") nếu tìm thấy,
        ngược lại trả về None.
    """
    if not isinstance(x, str):
        return None
    match = re.search(pattern_battery, x, flags=re.I)
    return f"{match.group(1)} Wh" if match else None

In [14]:
size["Thông tin Pin"] = size["Thông tin Pin"].apply(clean_1)
size["Thông tin Pin"].value_counts()

Thông tin Pin
41 Wh       47
54 Wh       32
60 Wh       28
42 Wh       26
75 Wh       22
50 Wh       21
70 Wh       18
57 Wh       16
58 Wh       15
90 Wh       15
59 Wh       14
80 Wh       12
53.5 Wh     11
52.4 Wh      9
76 Wh        8
45 Wh        8
56 Wh        7
43 Wh        7
72.4 Wh      7
99.9 Wh      6
63 Wh        5
41.04 Wh     5
53.8 Wh      4
65 Wh        4
52.5 Wh      4
47 Wh        4
96 Wh        4
48 Wh        4
64 Wh        4
53 Wh        3
57.5 Wh      3
51 Wh        3
99 Wh        2
39.3 Wh      2
72 Wh        2
99.98 Wh     2
38 Wh        2
46.8 Wh      2
83 Wh        2
86 Wh        2
55.2 Wh      2
66.5 Wh      1
40 Wh        1
54.8 Wh      1
74.5 Wh      1
40.2 Wh      1
42.3 Wh      1
73 Wh        1
78 Wh        1
71 Wh        1
Name: count, dtype: int64

In [15]:
size

,Thông tin Pin,Hệ điều hành,Thời điểm ra mắt,Kích thước,Chất liệu
pid,,,,,
327098,41 Wh,Windows 11 Home SL,2023,Dài 359.8 mm - Rộng 236 mm - Dày 18.6 mm - 1.59 kg,Vỏ nhựa
311178,42 Wh,Windows 11 Home SL,2023,Dài 360.3 mm - Rộng 232.5 mm - Dày 17.9 mm - 1.63 kg,Vỏ nhựa
334442,60 Wh,Windows 11 Home SL,2024,Dài 343.4 mm - Rộng 239.5 mm - Dày 17.9 mm - 1.63 kg,Vỏ nhựa - nắp lưng bằng kim loại
335362,53.8 Wh,macOS Sequoia,03/2025,Dài 304.1 mm - Rộng 215 mm - Dày 11.3 mm - 1.24 kg,Vỏ nhôm tái chế 100%
337470,41 Wh,Windows 11 Home SL + Office Home 2024 vĩnh viễn,2025,Dài 358.5 mm - Rộng 235.56 mm - Dày 18.99 mm - 1.9 kg,Vỏ nhựa
...,...,...,...,...,...
342508,57 Wh,Windows 11 Pro,2025,Dài 312.8 mm - Rộng 217.65 mm - Dày 18.3 mm - 1.22 kg,Vỏ nhôm
342517,80 Wh,Windows 11 Home SL + Office Home 2024 vĩnh viễn,2025,Dài 344.9 mm - Rộng 255.35 mm - Dày 21.54 mm - 1.9 kg,Vỏ nhựa - Nắp lưng bằng nhôm
342523,80 Wh,Windows 11 Home SL + Office Home 2024 vĩnh viễn,2025,Dài 344.9 mm - Rộng 255.35 mm - Dày 21.54 mm - 1.9 kg,Nắp lưng và chiếu nghỉ tay bằng nhôm


In [16]:
size['Hệ điều hành'].value_counts()

Hệ điều hành
Windows 11 Home SL                                                       256
Windows 11 Home SL + Office Home 2024 vĩnh viễn + Microsoft 365 Basic     44
Windows 11 Home SL + Office Home & Student 2021 vĩnh viễn                 42
Windows 11 Home SL + Office Home 2024 vĩnh viễn                           25
macOS Sequoia                                                             12
Windows 11 Pro                                                            11
Windows 11 Home SL + Office Home 2024 + Microsoft 365 Basic 1 năm          8
macOS                                                                      2
Chrome OS                                                                  2
Windows 11 Home SL + Microsoft 365 Personal                                1
Windows 11 Home + Office Home 2024 + Microsoft 365 Basic                   1
Windows 11 Home                                                            1
Name: count, dtype: int64

In [17]:
size[size['Hệ điều hành'] == "macOS Sequoia"]

,Thông tin Pin,Hệ điều hành,Thời điểm ra mắt,Kích thước,Chất liệu
pid,,,,,
335362,53.8 Wh,macOS Sequoia,03/2025,Dài 304.1 mm - Rộng 215 mm - Dày 11.3 mm - 1.24 kg,Vỏ nhôm tái chế 100%
335372,66.5 Wh,macOS Sequoia,03/2025,Dài 340.4 mm - Rộng 237.6 mm - Dày 11.5 mm - 1.51 kg,Vỏ nhôm tái chế 100%
331564,72.4 Wh,macOS Sequoia,10/2024,Dài 312.6 mm - Rộng 221.2 mm - Dày 15.5 mm - 1.55 kg,Vỏ nhôm tái chế 100%
331566,72.4 Wh,macOS Sequoia,10/2024,Dài 312.6 mm - Rộng 221.2 mm - Dày 15.5 mm - 1.55 kg,Vỏ nhôm tái chế 100%
331568,72.4 Wh,macOS Sequoia,10/2024,Dài 312.6 mm - Rộng 221.2 mm - Dày 15.5 mm - 1.6 kg,Vỏ nhôm tái chế 100%
332450,72.4 Wh,macOS Sequoia,10/2024,Dài 312.6 mm - Rộng 221.2 mm - Dày 15.5 mm - 1.6 kg,Vỏ nhôm tái chế 100%
342024,53.8 Wh,macOS Sequoia,03/2025,Dài 304.1 mm - Rộng 215 mm - Dày 11.3 mm - 1.24 kg,Vỏ nhôm tái chế 100%
332451,72.4 Wh,macOS Sequoia,10/2024,Dài 312.6 mm - Rộng 221.2 mm - Dày 15.5 mm - 1.62 kg,Vỏ nhôm tái chế 100%
335366,53.8 Wh,macOS Sequoia,03/2025,Dài 304.1 mm - Rộng 215 mm - Dày 11.3 mm - 1.24 kg,Vỏ nhôm tái chế 100%


In [18]:
df.link[df.index == 335362]

pid
335362    https://www.thegioididong.com/laptop/macbook-air-13-inch-m4-16gb-256gb
Name: link, dtype: object

In [19]:
size["Hệ điều hành"] = (
    size["Hệ điều hành"]
    .str.split("+", n=1).str[0]
    .str.strip()  
)

In [20]:
size["Hệ điều hành"].value_counts()

Hệ điều hành
Windows 11 Home SL    376
macOS Sequoia          12
Windows 11 Pro         11
macOS                   2
Windows 11 Home         2
Chrome OS               2
Name: count, dtype: int64

In [21]:
size.columns

Index(['Thông tin Pin', 'Hệ điều hành', 'Thời điểm ra mắt', 'Kích thước',
       'Chất liệu'],
      dtype='object')

In [22]:
size["Thời điểm ra mắt"].value_counts()

Thời điểm ra mắt
2024       153
2025       151
2023        63
2022         9
10/2024      7
03/2025      5
6/2022       1
2021         1
2020         1
Name: count, dtype: int64

In [23]:
size["Thời điểm ra mắt"] = (
    size["Thời điểm ra mắt"]
    .str.split("/", n=1).str[-1]
    .str.strip()  
)
size['Thời điểm ra mắt'].value_counts()

Thời điểm ra mắt
2024    160
2025    156
2023     63
2022     10
2021      1
2020      1
Name: count, dtype: int64

In [24]:
size["Cân nặng"] = (
    size["Kích thước"]
    .str.split("-",).str[-1]
    .str.strip()
)
size["Cân nặng"].value_counts()

Cân nặng
1.7 kg      38
1.9 kg      22
1.66 kg     21
1.5 kg      16
1.39 kg     15
            ..
1.145 kg     1
1.326 kg     1
2.533 kg     1
1.327 kg     1
2 kg         1
Name: count, Length: 100, dtype: int64

In [25]:
def round(x):
    """
    Làm tròn gần nhất số cân nặng lên 1 chữ số thập phân.

    Parameters
    ----------
    x : str
        Chuỗi cân nặng, dạng '<số> kg', ví dụ '1.66 kg'.

    Returns
    -------
    str
        Chuỗi cân nặng đã làm tròn gần nhất 1 chữ số thập phân, ví dụ '1.7 kg'.

    Examples
    --------
    >>> round_weight("1.66 kg")
    '1.7 kg'
    >>> round_weight("1.145 kg")
    '1.1 kg'
    """
    num, unit = x.split()
    # làm tròn lên 1 chữ số thập phân
    rounded = np.round(float(num) * 10) / 10
    return f"{rounded} {unit}"

size["Cân nặng"] = size["Cân nặng"].apply(round)
size["Cân nặng"].value_counts()

Cân nặng
1.7 kg    77
1.4 kg    47
1.6 kg    45
1.9 kg    39
1.5 kg    30
2.2 kg    24
1.2 kg    20
2.3 kg    18
1.3 kg    18
2.4 kg    17
1.8 kg    13
2.1 kg    13
2.5 kg    12
2.0 kg    10
1.0 kg     8
2.6 kg     3
2.7 kg     2
2.8 kg     2
3.6 kg     2
1.1 kg     2
3.5 kg     2
2.9 kg     1
Name: count, dtype: int64

In [26]:
size.columns

Index(['Thông tin Pin', 'Hệ điều hành', 'Thời điểm ra mắt', 'Kích thước',
       'Chất liệu', 'Cân nặng'],
      dtype='object')

In [27]:
try:
    size.drop(columns= "Kích thước", inplace= True)
except:
    None
size.to_csv(r"~\Jupyter Notebook\clean data\size_clean.csv", index= True)

In [28]:
size

,Thông tin Pin,Hệ điều hành,Thời điểm ra mắt,Chất liệu,Cân nặng
pid,,,,,
327098,41 Wh,Windows 11 Home SL,2023,Vỏ nhựa,1.6 kg
311178,42 Wh,Windows 11 Home SL,2023,Vỏ nhựa,1.6 kg
334442,60 Wh,Windows 11 Home SL,2024,Vỏ nhựa - nắp lưng bằng kim loại,1.6 kg
335362,53.8 Wh,macOS Sequoia,2025,Vỏ nhôm tái chế 100%,1.2 kg
337470,41 Wh,Windows 11 Home SL,2025,Vỏ nhựa,1.9 kg
...,...,...,...,...,...
342508,57 Wh,Windows 11 Pro,2025,Vỏ nhôm,1.2 kg
342517,80 Wh,Windows 11 Home SL,2025,Vỏ nhựa - Nắp lưng bằng nhôm,1.9 kg
342523,80 Wh,Windows 11 Home SL,2025,Nắp lưng và chiếu nghỉ tay bằng nhôm,1.9 kg
